**Descripción del proyecto**

Los clientes de Beta Bank se están yendo, cada mes, poco a poco. Los banqueros descubrieron que es más barato salvar a los clientes existentes que atraer nuevos.
Necesitamos predecir si un cliente dejará el banco pronto. Tú tienes los datos sobre el comportamiento pasado de los clientes y la terminación de contratos con el banco.
Crea un modelo con el máximo valor F1 posible. Para aprobar la revisión, necesitas un valor F1 de al menos 0.59. Verifica F1 para el conjunto de prueba.
Además, debes medir la métrica AUC-ROC y compararla con el valor F1.

### Tabla de Contenidos
**Objetivos:**
- 1. Descargar y prepar los datos
- 2. Examinar el equilibrio de las clases, entrenar el modelo sin tener en cuenta el desequilibrio y describir brevemente los hallazgos
- 3. Mejorar la calidad del modelo. Utilizar al menos dos enfoques para corregir el desequilibrio de clases.
- 4. Realizar la prueba final.
- 5. Conclusión general

## Descargamos y preparamos los datos

In [ ]:
# Cargamos las librerías necesarias
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OrdinalEncoder
from sklearn.metrics import (
roc_auc_score, f1_score)

In [ ]:
# Cargamos el DataFrame
df = pd.read_csv("/datasets/Churn.csv")

In [ ]:
# Revisamos el DataFrame
df.head(10)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0
5,6,15574012,Chu,645,Spain,Male,44,8.0,113755.78,2,1,0,149756.71,1
6,7,15592531,Bartlett,822,France,Male,50,7.0,0.00,2,1,1,10062.80,0
7,8,15656148,Obinna,376,Germany,Female,29,4.0,115046.74,4,1,0,119346.88,1
8,9,15792365,He,501,France,Male,44,4.0,142051.07,2,0,1,74940.50,0
9,10,15592389,H?,684,France,Male,27,2.0,134603.88,1,1,1,71725.73,0


In [ ]:
# Revisamos información general del DataFrame
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           9091 non-null   float64
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


El DataFrame consta de 10000 filas y 14 columnas, donde **Exited** es nuestra variable objetivo.

A contunuación revisaremos si hay duplicados y valores ausentes.

In [ ]:
# Revisamos duplicados
df.duplicated().sum()

0

In [ ]:
# Revisamos valores ausentes
df.isna().sum()

RowNumber            0
CustomerId           0
Surname              0
CreditScore          0
Geography            0
Gender               0
Age                  0
Tenure             909
Balance              0
NumOfProducts        0
HasCrCard            0
IsActiveMember       0
EstimatedSalary      0
Exited               0
dtype: int64

In [ ]:
# Calculamos porcentaje de valores ausentes
t_nan = df.Tenure.isna().sum() / len(df.Tenure)*100
print(f"porcentaje de NaN en 'Tenure': {int(t_nan)}%")

porcentaje de NaN en 'Tenure': 9%


In [ ]:
# Revisamos la distribución de la columna
df.Tenure.describe()

count    9091.000000
mean        4.997690
std         2.894723
min         0.000000
25%         2.000000
50%         5.000000
75%         7.000000
max        10.000000
Name: Tenure, dtype: float64

In [ ]:
df.Tenure.value_counts()

1.0     952
2.0     950
8.0     933
3.0     928
5.0     927
7.0     925
4.0     885
9.0     882
6.0     881
10.0    446
0.0     382
Name: Tenure, dtype: int64

En este caso la columna tiene un 9% de valores ausentes. Los imputaremos con la mediana de la columna debido a la variabilidad de los valores.

In [ ]:
# Imputamos con la mediana
t_median = df.Tenure.median()
df.Tenure.fillna(t_median, inplace=True)

In [ ]:
# Revisamos que esté todo en orden
df.Tenure.isna().sum()

0

En este caso utilizaremos la **OrdinalEncoder** ya que tenemos columnas del tipo *object* (**Geography** y **Gender**) y entrenaremos un modelo RandomForestClassifier.

In [ ]:
# Utilizamos la codificación de etiquetas
encoder = OrdinalEncoder()
df[["Geography", "Gender"]] = encoder.fit_transform(df[["Geography", "Gender"]])

In [ ]:
# Revisamos los nuevos valores
df.head(10)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,0.0,0.0,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,2.0,0.0,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,0.0,0.0,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,0.0,0.0,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,2.0,0.0,43,2.0,125510.82,1,1,1,79084.10,0
5,6,15574012,Chu,645,2.0,1.0,44,8.0,113755.78,2,1,0,149756.71,1
6,7,15592531,Bartlett,822,0.0,1.0,50,7.0,0.00,2,1,1,10062.80,0
7,8,15656148,Obinna,376,1.0,0.0,29,4.0,115046.74,4,1,0,119346.88,1
8,9,15792365,He,501,0.0,1.0,44,4.0,142051.07,2,0,1,74940.50,0
9,10,15592389,H?,684,0.0,1.0,27,2.0,134603.88,1,1,1,71725.73,0


**Ahora dividiremos el conjunto de datos en un conjunto de entrenamiento (80%) y un conjunto de prueba (20%).**

- **Exited** en nuestra variable objetivo.
- En este caso "Exited", "Surname", "CustomerId" y "RowNumber" **NO** los consideraremos como features.


In [ ]:
# Definimos el Target y los Features
target = "Exited"

features = df.drop(columns=["Exited", "Surname", "CustomerId", "RowNumber" ]).columns.values

## Examinamos el equilibrio de las clases, entrenamos el modelo sin tener en cuenta el desequilibrio y describimos brevemente los hallazgos.

In [ ]:
# Examinamos el equilibrio de las clases
df.Exited.value_counts(normalize=True)

0    0.7963
1    0.2037
Name: Exited, dtype: float64

En este caso el 20% de los clientes han abandonado este banco (20% de los valores son 1).

Ahora dividiremos el conjunto de datos en un conjunto de entrenamiento (80%) y un conjunto de prueba (20%).

In [ ]:
# Dividimos el conjunto de datos
random_state = 42

X_full_train, X_test, y_full_train, y_test = train_test_split(df[features],
                                                             df[target],
                                                             test_size=0.2,
                                                             random_state=random_state)

In [ ]:
# Entrenamos un modelo RandomForestClassifier
model = RandomForestClassifier(random_state=random_state)
model.fit(X_full_train, y_full_train)

#predictions = model.predict(X_test)
predictions = model.predict_proba(X_test)[:,1]

In [ ]:
# Visualizamos el rendimiento del modelo de acuerdo a las métricas F1 Score y ROC AUC.
print(F"""
F1 Score: {f1_score(y_test, predictions >= 0.5)}
ROC AUC: {roc_auc_score(y_test, predictions)}
""")


F1 Score: 0.5772230889235569
ROC AUC: 0.8576749937851417



- El rendimiento de acuerdo al valor ROC AUC es de 0.85, bastante cercano a 1. En este caso el valor es más alto debido a que hay un desequilibrio en los datos.

- En el caso de F1 Score el rendimiento es de 0.57. Al contrario, este tiende a bajar cuando hay un desequilibrio en los datos.

## Mejoramos la calidad del modelo. Utilizaremos al menos dos enfoques para corregir el desequilibrio de clases.
Además utilizaremos conjuntos de entrenamiento y validación para encontrar el mejor modelo y el mejor conjunto de parámetros. También entrenaremos diferentes modelos en los conjuntos de entrenamiento y validación para encontrar el mejor.

In [ ]:
# Creamos una muestra de validación
X_train, X_valid, y_train, y_valid = train_test_split(X_full_train,
                                                      y_full_train,
                                                      test_size=0.2,
                                                      random_state=random_state)

### Ajustamos el peso de las clases

Probaremos distintos hiperparámetros para "class_weight"

In [ ]:
%%time
adjusted_model = RandomForestClassifier(random_state=random_state, class_weight="balanced")
adjusted_model.fit(X_train, y_train)

adjusted_predictions = adjusted_model.predict_proba(X_valid)[:,1]


CPU times: user 1.04 s, sys: 12.1 ms, total: 1.06 s
Wall time: 1.07 s


In [ ]:
print(F"""
F1 Score: {f1_score(y_valid, adjusted_predictions >= 0.5)}
ROC AUC: {roc_auc_score(y_valid, adjusted_predictions)}
""")


F1 Score: 0.5831775700934579
ROC AUC: 0.8570918944629822



In [ ]:
%%time
adjusted_sub_model = RandomForestClassifier(random_state=random_state, class_weight="balanced_subsample")
adjusted_sub_model.fit(X_train, y_train)

adjusted_sub_predictions = adjusted_sub_model.predict_proba(X_valid)[:,1]

CPU times: user 1.21 s, sys: 7.88 ms, total: 1.22 s
Wall time: 1.23 s


In [ ]:
print(F"""
F1 Score: {f1_score(y_valid, adjusted_sub_predictions >= 0.5)}
ROC AUC: {roc_auc_score(y_valid, adjusted_sub_predictions)}
""")


F1 Score: 0.5897920604914935
ROC AUC: 0.8553190109327818



- Al ajustar el peso de las clases obtenemos resultados ligeramente diferentes.
- Al utilizar el conjunto de validación obtenemos resultados de las métricas muy similares en relación a los resultados del conjunto de prueba.

### Cambiamos el umbral

In [ ]:
%%time
threshold_model = RandomForestClassifier(random_state=random_state)
threshold_model.fit(X_train, y_train)

threshold_predictions = threshold_model.predict_proba(X_valid)[:,1]

CPU times: user 1.06 s, sys: 12.2 ms, total: 1.07 s
Wall time: 1.07 s


In [ ]:
# Buscamos el mejor umbral para la métrica F1
best_threshold = 0
best_f1_score = 0

for t in np.linspace(0, 1, 101):
    f1_score_tmp = f1_score(y_valid, threshold_predictions >= t)
    if f1_score_tmp > best_f1_score:
        best_f1_score = f1_score_tmp
        best_threshold = t

print(f"Best Threshold: {best_threshold}")

Best Threshold: 0.38


In [ ]:
print(F"""
F1 Score: {f1_score(y_valid, threshold_predictions >= best_threshold)}
ROC AUC: {roc_auc_score(y_valid, threshold_predictions)}
""")


F1 Score: 0.6224328593996841
ROC AUC: 0.8527245472300496



Cómo se puede ver, al modificar el umbral el rendimiento respecto a F1 Score, mejora. Sube de 0.58 a 0.62.

## Realizamos la prueba final

En este caso no modificaremos **class_weight** debido a que el cambio generado es minúsculo.

- Realizamos una prueba final con el conjunto de validación.

In [ ]:
%%time
tuned_model = RandomForestClassifier(random_state=random_state)
tuned_model.fit(X_train, y_train)

tuned_valid_predictions = tuned_model.predict_proba(X_valid)[:,1]

CPU times: user 1.05 s, sys: 12 ms, total: 1.06 s
Wall time: 1.08 s


In [ ]:
# Buscamos el mejor umbral para la métrica F1
best_threshold = 0
best_f1_score = 0

for t in np.linspace(0, 1, 101):
    f1_score_tmp = f1_score(y_valid, tuned_valid_predictions >= t)
    if f1_score_tmp > best_f1_score:
        best_f1_score = f1_score_tmp
        best_threshold = t

print(f"Best Threshold: {round(best_threshold, 4)}")

Best Threshold: 0.38


In [ ]:
# Resultados para el conjunto de validación
print(F"""
Validation Results:
F1 Score: {f1_score(y_valid, tuned_valid_predictions >= best_threshold)}
ROC AUC: {roc_auc_score(y_valid, tuned_valid_predictions)}
""")


Validation Results:
F1 Score: 0.6224328593996841
ROC AUC: 0.8527245472300496



### Modelo Final

In [ ]:
%%time
final_model = RandomForestClassifier(random_state=random_state)
final_model.fit(X_full_train, y_full_train)

final_test_predictions = final_model.predict_proba(X_test)[:,1]

CPU times: user 1.32 s, sys: 19.8 ms, total: 1.34 s
Wall time: 1.36 s


In [ ]:
# Resultados para el conjunto de prueba
print(F"""
Test Results:
F1 Score: {f1_score(y_test, final_test_predictions >= best_threshold)}
ROC AUC: {roc_auc_score(y_test, final_test_predictions)}
""")

final_test_predictions = final_model.predict_proba(X_test)[:,1]


Test Results:
F1 Score: 0.6201342281879195
ROC AUC: 0.8576749937851417



Para nuestro modelo final y luego de hacer algunas pruebas y ajustes obtuvimos los siguientes rendimientos.

- f1 Score: 0.62
- ROC AUC: 0.85

## Conclución general

1.- En este estudio:
- 1. Descargamos y prepamos los datos
- 2. Examinamos el equilibrio de las clases, entrenamos el modelo sin tener en cuenta el desequilibrio.
- 3. Mejoramos la calidad del modelo. Utilizamos al menos dos enfoques para corregir el desequilibrio de clases.
- 4. Realizamos la prueba final.

2.- Al examinar el equilibrio de clases:
 -  el 20% de los clientes abandonó el banco (20% de los valores son 1)
 -  Al entrenar el modelo:
  - El rendimiento de acuerdo al valor ROC AUC es de 0.85, bastante cercano a 1. En este caso el valor es más alto debido a que hay un desequilibrio en los datos.

  - En el caso de F1 Score el rendimiento es de 0.57. Al contrario, este tiende a bajar cuando hay un desequilibrio en los datos.

3.- Al ajustar el peso de las clases:
 - Hubo un cambio ligero en el rendimiento de acuerdo a las métricas.

4.- Al modificar el umbral:
 - El rendimiento respecto a F1 Score, mejora. Sube de 0.58 a 0.62.

5.- Al realizar el modelo final obtuvimos los siguientes rendimientos:
 - f1 Score: 0.62
 - ROC AUC: 0.85
  - En este caso el valor de ROC AUC fue mayor debido al desequilibrio de clases. Por el contrario, el valor de f1 Score fue menor debido al mismo motivo.